In [4]:
%pip install numpy pandas matplotlib seaborn sklearn scikit starlette pickle scipy mlflow pickle joblib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [5]:
import logging
import sys
import warnings
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [6]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [7]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [8]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [9]:
csv_url = (
        "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
    )

In [10]:
data = pd.read_csv(csv_url, sep=";")

In [11]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [12]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


In [16]:
with mlflow.start_run():
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    lr = ElasticNet(alpha=0.55, l1_ratio=0.55, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print(f"Elasticnet model (alpha={0.55:f}, l1_ratio={0.55:f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

    mlflow.log_param("alpha", 0.55)
    mlflow.log_param("l1_ratio", 0.55)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    predictions = lr.predict(train_x)

    signature = infer_signature(train_x, predictions)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    mlflow.sklearn.log_model(lr, "model")

    # # Model registry does not work with file store
    # if tracking_url_type_store != "file":
    #     # Register the model
    #     # There are other ways to use the Model Registry, which depends on the use case,
    #     # please refer to the doc for more information:
    #     # https://mlflow.org/docs/latest/model-registry.html#api-workflow
    #     mlflow.sklearn.log_model(
    #         lr, "model", registered_model_name="ElasticnetWineModel", signature=signature
    #     )
    # else:
    #     mlflow.sklearn.log_model(lr, "model", signature=signature)


Elasticnet model (alpha=0.550000, l1_ratio=0.550000):
  RMSE: 0.7195723367943433
  MAE: 0.5927151780442308
  R2: 0.09160640722864588


In [18]:
# load the model
logged_model = 'runs:/e456d55690dd4ef19fec938fb15dcfeb/model'
model = mlflow.sklearn.load_model(logged_model)

In [20]:
predicted_qualities = model.predict(test_x)
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
(rmse, mae, r2)


(0.7195723367943433, 0.5927151780442308, 0.09160640722864588)